<a href="https://colab.research.google.com/github/plp940/Temp-Prediction/blob/main/Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#prepare a cnn model for the prediction of lung cancer
#importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf
#import os #for uploading the zip dataset from the session storage
from tensorflow import keras
from tensorflow.keras.models import Sequential#to create a stack of layers or model
from tensorflow.keras.layers import LSTM,Dense#preperation of cnn model
from sklearn.model_selection import train_test_split
#from zipfile import ZipFile#to extract the zipped data into a folder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [4]:
import joblib

In [5]:
df = pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"],index_col="Date")
df.head()

/tmp/ipython-input-5-3051996484.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"],index_col="Date")


,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [6]:
df.isna().sum(axis=0)

,0
Temp,0


In [7]:
#not before it should be after
#df = df.dropna()

In [8]:
#string format for a numeric value
#temp = "hello"
#temp = NaN

#converting the sting values into numreric values ,if the string value does not make any sense then we convert that value into null NaN
df["Temp"] = pd.to_numeric(df["Temp"],errors="coerce")

In [9]:
df = df.dropna()

In [10]:
#normalize the features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df["Temp"].values.reshape(-1,1))

In [12]:
# ✅ Save the scaler using pickle
import pickle
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [13]:
#create the sequence model from rnn model
#sequence length for temperatue
sequence_length = 30


In [14]:
#function for creating sequences
def create_sequences(data_scaled, sequence_length):
    X, y = [], []
    for i in range(len(data_scaled) - sequence_length):
        X.append(data_scaled[i:i + sequence_length])
        y.append(data_scaled[i + sequence_length])
    return np.array(X), np.array(y)


In [15]:
X,y=create_sequences(data_scaled,sequence_length)

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)

In [17]:
model = Sequential([
    LSTM(64,activation="relu",input_shape=(sequence_length,1)),
    Dense(1)#output layer is single value
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(optimizer="adam",loss="mse",metrics=["accuracy"])
#we are predicting sequential data in degree celcius -regression problem so not binary crossentropy,categorical crossentrpy

In [19]:
model.fit(X_train,y_train,epochs=20,batch_size=32)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 3.4633e-04 - loss: 0.0870
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0026 - loss: 0.0116
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0016 - loss: 0.0109
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 3.6646e-04 - loss: 0.0107
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0015 - loss: 0.0103
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 6.6097e-04 - loss: 0.0101
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 7.1211e-04 - loss: 0.0099
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 8.5293e-04 - loss: 0.0098
Epoch 9/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 9.4212e-04 - loss: 0.0098
Epoch 10/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0019 - loss: 0.0097
Epoch 11/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0011 - loss: 0.0091
Epoch 12/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 

In [20]:
y_pred_scaled =model.predict(X_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [21]:
#inverse transform
y_pred_scaled=np.clip(y_pred_scaled,0,1)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_inverse = scaler.inverse_transform(y_test)

In [22]:
#we trained model now testing on random data
#predicting next day temperature
last_sequence = data_scaled[-sequence_length:].reshape(1, sequence_length, 1)
next_day_temp_scaled = model.predict(last_sequence)
next_day_temp = np.clip(next_day_temp_scaled, 0, 1)
next_day_temp = scaler.inverse_transform(next_day_temp)
print("the next day temp is",next_day_temp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
the next day temp is [[12.623761]]


In [23]:
model.save("temperature.h5")